# BIOfid DISeg 
## Document Image Segmentation



### Configuration

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import biofid

%matplotlib inline 

tf.logging.set_verbosity(tf.logging.ERROR)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = biofid.BIOfidConfig()
BIOFID_DIR = os.path.join("../data/")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# config.display()

DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

/home/stud_homes/s3676959/anaconda3/envs/DISeg/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stud_homes/s3676959/anaconda3/envs/DISeg/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stud_homes/s3676959/anaconda3/envs/DISeg/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stud

### Load Model and Data

In [2]:
# Load validation dataset
dataset = biofid.BIOfidDataset()
dataset.load_dataset(BIOFID_DIR, "val")

# Must call before using the dataset
dataset.prepare()

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

weights_path = '../logs/biofid20191007T1636/mask_rcnn_biofid_0100.h5'
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  ../logs/biofid20191007T1636/mask_rcnn_biofid_0100.h5
Re-starting from epoch 100


## Validation Set Examples
### Run Detection

In [3]:
images = []
for image_id in range(10):
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    images.append(image)

results = [model.detect([image])[0] for image in images]
C = visualize.random_colors(10)

### Display Samples

In [4]:
def display_sample(image_id = 0):
    r = results[image_id]
    image = images[image_id]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], title="Predictions",
                                colors=C)

interact(display_sample, image_id=widgets.IntSlider(min=0, max=9, step=1, value=0));

interactive(children=(IntSlider(value=0, description='image_id', max=9), Output()), _dom_classes=('widget-inte…